<a href="https://colab.research.google.com/github/Cosima10/master-thesis/blob/main/keyword_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## Preparation

---


#### <span style="color:pink"> USE THIS MODULE IF YOU ARE RUNNING IT IN GOOGLE COLAB </span>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
!pip install selenium
from selenium import webdriver
import sys
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
# !pip install selenium, webdriver_manager
# !apt-get update  # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp / usr/lib/chromium-browser/chromedriver / usr/bin
!apt install chromium-chromedriver
!cp / usr/lib/chromium-browser/chromedriver / usr/bin
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver = wd
import os

path_source = "/content/drive/MyDrive/scraper/df link.xlsx"
path_output = "/content/drive/MyDrive/scraper/result/"
if not os.path.exists(path_output):
    os.makedirs(path_output)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (101.0.4951.64-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
cp: target 'usr/bin' is not a directory


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


#### <span style="color:pink"> USE THIS MODULE IF YOU ARE RUNNING IT ON YOUR LOCAL MACHINE </span>

In [8]:
# !pip install selenium, webdriver_manager, numpy, pandas
# import time
# import numpy as np
# from bs4 import BeautifulSoup
# import pandas as pd
# import sys
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# import os
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# wd = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
# driver = wd
# cwd = os.getcwd()
# cwd

# path_source = cwd+"/df link.xlsx"
# path_output = cwd+"/result/"
# if not os.path.exists(path_output):
#     os.makedirs(path_output)


---

# SCRAPER

---

In [4]:
# initialising string

def hapusspecialcharacter(ini_string):
    getVals = list([val for val in ini_string
                    if val.isalpha() or val.isnumeric() or val == " "])
    result = "".join(getVals)
    return result


def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)


def years_check(year, df):
    number_of_news_in_year = len(df[df['date'].str.contains(str(year))])
    return number_of_news_in_year

df_link = pd.read_excel(path_source)


In [5]:
# create output folders and files 

def createExcelFiles(iprocess, kamus, keyword):
    if not os.path.exists(f"{path_output}/{df_link['country'][iprocess]}"):
        os.makedirs(f"{path_output}/{df_link['country'][iprocess]}")

    results_path = f"{path_output}/{df_link['country'][iprocess]}/"

    if not os.path.exists(f"{results_path}/{keyword}"):
        os.makedirs(f"{results_path}/{keyword}")

    results_path = f"{results_path}/{keyword}/"
    
    df = pd.DataFrame(kamus).astype(str)
    file_name = f"{results_path}{df_link['country'][iprocess]} {df_link['number'][iprocess]} {df_link['link'][iprocess].split('/')[2].replace('.','_')}.xlsx"
    df.to_excel(file_name, index=False)

    yearList = []
    counterList = []
    for i in range(2023)[-6:]:
        yearList.append(i)
        counterList.append(years_check(i, df))

    file_nameCek = f"{results_path}{keyword} {df_link['country'][iprocess]} {df_link['number'][iprocess]} {df_link['link'][iprocess].split('/')[2].replace('.','_')}.xlsx"
    kamusCek = {"year": yearList, "Number of Related News": counterList}
    dfCek = pd.DataFrame(kamusCek).astype(str)
    dfCek.to_excel(file_nameCek, index=False)
    return dfCek


---

## DECORATOR
replacing the common part of each scraper and therewith reducing redundancy

---

In [6]:
def scrap_data(params):
    def scrap_decorator(fn):
        iprocess = params.get('iprocess')
        url = params.get('url')
        pagination = params.get('pagination')

        def wrapper(*args, **kwargs):
            keyword = args[0] if args else kwargs.get('keyword', None)
            news_links = []
            dates = []
            # for ipage in list(range(pagination))[1:]:
            for ipage in pagination:
                wd.get(url.format(keyword=keyword, ipage=ipage))  # link with search query
                content = driver.page_source.encode('utf-8').strip()
                soup = BeautifulSoup(content)
                news_links, dates, found = fn(soup, ipage, news_links, dates)
                if not found:
                    print(f"[PAGE NO : {ipage}] [NO MORE ARTICLES FOUND]")
                    break
                
            kamus = {"news link": news_links, "date": dates}
            return createExcelFiles(iprocess, kamus, keyword)

        return wrapper
    return scrap_decorator

---

# GERMANY

---

#### 1) NTV

In [7]:
@scrap_data(params={'iprocess': 7, 'url': 'https://www.ntv.de/suche/?q={keyword}&at=all&page={ipage}', 'pagination': list(range(1,21))})
def germany1ntv(soup, ipage, news_links, dates):
    found = True
    b = soup.find('div', class_='search__results').find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("span", "teaser__date").text.split()))
    return news_links, dates, found

In [15]:
#print(germany1ntv("dsgvo"))

[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 20]
[PAGE NO : 6] [ARTICLES FOUND: 20]
[PAGE NO : 7] [ARTICLES FOUND: 20]
[PAGE NO : 8] [ARTICLES FOUND: 20]
[PAGE NO : 9] [ARTICLES FOUND: 14]
[PAGE NO : 10] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      3
1  2018                     91
2  2019                     34
3  2020                     15
4  2021                     23
5  2022                      7


In [16]:
keyword_list = ["gdpr", "dsgvo", "datenschutz", "schrems", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in keyword_list:
  print(germany1ntv(keyword))

[PAGE NO : 1] [ARTICLES FOUND: 3]
[PAGE NO : 2] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      1
3  2020                      1
4  2021                      1
5  2022                      0
[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 20]
[PAGE NO : 6] [ARTICLES FOUND: 20]
[PAGE NO : 7] [ARTICLES FOUND: 20]
[PAGE NO : 8] [ARTICLES FOUND: 20]
[PAGE NO : 9] [ARTICLES FOUND: 14]
[PAGE NO : 10] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      3
1  2018                     91
2  2019                     34
3  2020                     15
4  2021                     23
5  2022                      7
[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20

#### 2) Deutsche Welle

In [ ]:
@scrap_data(params={'iprocess': 9, 'url': 'https://www.dw.com/search/?languageCode=en&item={keyword}&searchNavigationId=9097&sort=DATE&resultsCounter=10000', 'pagination': list(range(1,21))})
def germany2dw(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "searchResult")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("span", "date").text.split()))
    return news_links, dates, found

In [ ]:
print(germany2dw("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 53]
[PAGE NO : 2] [ARTICLES FOUND: 53]
[PAGE NO : 3] [ARTICLES FOUND: 53]
[PAGE NO : 4] [ARTICLES FOUND: 53]
[PAGE NO : 5] [ARTICLES FOUND: 53]
[PAGE NO : 6] [ARTICLES FOUND: 53]
[PAGE NO : 7] [ARTICLES FOUND: 53]
[PAGE NO : 8] [ARTICLES FOUND: 53]
[PAGE NO : 9] [ARTICLES FOUND: 53]
[PAGE NO : 10] [ARTICLES FOUND: 53]
[PAGE NO : 11] [ARTICLES FOUND: 53]
[PAGE NO : 12] [ARTICLES FOUND: 53]
[PAGE NO : 13] [ARTICLES FOUND: 53]
[PAGE NO : 14] [ARTICLES FOUND: 53]
[PAGE NO : 15] [ARTICLES FOUND: 53]
[PAGE NO : 16] [ARTICLES FOUND: 53]
[PAGE NO : 17] [ARTICLES FOUND: 53]
[PAGE NO : 18] [ARTICLES FOUND: 53]
[PAGE NO : 19] [ARTICLES FOUND: 53]
[PAGE NO : 20] [ARTICLES FOUND: 53]
   year Number of Related News
0  2017                      0
1  2018                    560
2  2019                    140
3  2020                    220
4  2021                    140
5  2022                      0


#### 3) Frankfurter Allgemeine Zeitung

In [ ]:
@scrap_data(params={'iprocess': 10, 'url': 'https://www.faz.net/suche/s{ipage}.html?ct=article&ct=audio&ct=blog&ct=gallery&ct=infografik&ct=storytelling&ct=video&&query={keyword}#listPagination', 'pagination': list(range(1,21))})
def germany3faz(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("li", "lst-Teaser_Item")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found


In [ ]:
print(germany3faz("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 8]
[PAGE NO : 2] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      1
1  2018                      4
2  2019                      2
3  2020                      1
4  2021                      0
5  2022                      0


---

# DENMARK

---

#### 1) INFORMATION.DK

In [ ]:
@scrap_data(params={'iprocess': 0, 'url': 'https://www.information.dk/search/site/{keyword}?page={ipage}', 'pagination': list(range(1,21))})
def denmark1information(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", {"typeof": "sioc:Item foaf:Document"})
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://www.information.dk"+i.find("a").get("href"))
        dates.append(" ".join(i.find("li", "date first").text.split()))
    return news_links, dates, found


In [ ]:
print(denmark1information("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 20]
[PAGE NO : 6] [ARTICLES FOUND: 8]
[PAGE NO : 7] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      1
1  2018                     27
2  2019                     34
3  2020                     27
4  2021                     18
5  2022                      0


#### 2) POLITIKEN.DK

In [ ]:
@scrap_data(params={'iprocess': 1, 'url': 'https://politiken.dk/search/?q={keyword}&target=pol&sort=pd&page={ipage}', 'pagination': list(range(1,21))})
def denmark2politiken(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "search-result__article u-padding--vertical-normal")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://politiken.dk/"+i.find("a").get("href"))
        dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

In [ ]:
print(denmark2politiken("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 20]
[PAGE NO : 6] [ARTICLES FOUND: 20]
[PAGE NO : 7] [ARTICLES FOUND: 20]
[PAGE NO : 8] [ARTICLES FOUND: 20]
[PAGE NO : 9] [ARTICLES FOUND: 20]
[PAGE NO : 10] [ARTICLES FOUND: 20]
[PAGE NO : 11] [ARTICLES FOUND: 11]
[PAGE NO : 12] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      3
1  2018                     54
2  2019                     56
3  2020                     42
4  2021                     44
5  2022                     12


#### 3) BERLINGSKE.DK

In [ ]:
@scrap_data(params={'iprocess': 4, 'url': 'https://www.berlingske.dk/search?query={keyword}&limit=10&offset={ipage}0', 'pagination': list(range(1,21))})
def denmark3berlingske(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "teaser__description align-self-top w-100")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://www.berlingske.dk/"+i.find("a").get("href"))
        dates.append(" ".join(i.find("div","teaser__date").text.split()))
    return news_links, dates, found

In [ ]:
print(denmark3berlingske("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 10]
[PAGE NO : 2] [ARTICLES FOUND: 10]
[PAGE NO : 3] [ARTICLES FOUND: 10]
[PAGE NO : 4] [ARTICLES FOUND: 10]
[PAGE NO : 5] [ARTICLES FOUND: 10]
[PAGE NO : 6] [ARTICLES FOUND: 10]
[PAGE NO : 7] [ARTICLES FOUND: 10]
[PAGE NO : 8] [ARTICLES FOUND: 10]
[PAGE NO : 9] [ARTICLES FOUND: 10]
[PAGE NO : 10] [ARTICLES FOUND: 10]
[PAGE NO : 11] [ARTICLES FOUND: 10]
[PAGE NO : 12] [ARTICLES FOUND: 10]
[PAGE NO : 13] [ARTICLES FOUND: 10]
[PAGE NO : 14] [ARTICLES FOUND: 10]
[PAGE NO : 15] [ARTICLES FOUND: 10]
[PAGE NO : 16] [ARTICLES FOUND: 10]
[PAGE NO : 17] [ARTICLES FOUND: 10]
[PAGE NO : 18] [ARTICLES FOUND: 10]
[PAGE NO : 19] [ARTICLES FOUND: 10]
[PAGE NO : 20] [ARTICLES FOUND: 10]
   year Number of Related News
0  2017                      0
1  2018                     16
2  2019                     58
3  2020                     53
4  2021                     63
5  2022                     10


#### 4) JV.DK

In [ ]:
@scrap_data(params={'iprocess': 2, 'url': 'https://jv.dk/soeg?keywords={keyword}&limit=10&offset={ipage}0', 'pagination': list(range(1,21))})
def denmark4jv(soup, ipage, news_links, dates):
    found = True
    b = soup.find('div', class_='page-block').find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://jv.dk/"+i.find("a").get("href"))
        dates.append(" ".join(i.find("time", "search-result__date").text.split()))
    return news_links, dates, found

In [ ]:
print(denmark4jv("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 6]
[PAGE NO : 2] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      0
3  2020                      0
4  2021                      0
5  2022                      6


---

# AUSTRIA

---

#### 1) WIENER ZEITUNG

In [ ]:
@scrap_data(params={'iprocess': 23, 'url': 'https://www.wienerzeitung.at/suche/?such={keyword}&date_min=20220112&date_max=20220412&em_page={ipage}', 'pagination': list(range(1,21))})
def austria1wienerzeitung(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "col-12 col-md-4 em_mobile_list_layout")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

In [ ]:
print(austria1wienerzeitung("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 12]
[PAGE NO : 2] [ARTICLES FOUND: 12]
[PAGE NO : 3] [ARTICLES FOUND: 8]
[PAGE NO : 4] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      0
3  2020                      0
4  2021                      0
5  2022                     32


#### 2) KLEINE ZEITUNG

In [ ]:
@scrap_data(params={'iprocess': 24, 'url': 'https://www.kleinezeitung.at/user/search.do?action=1&resultsPage={ipage}&resetForm=0&type=4196%2C4193%2C4206&searchText={keyword}', 'pagination': list(range(1,21))})
def austria2kleinezeitung(soup, ipage, news_links, dates):
    found = True
    b = soup.find('section', class_='article-card--search').find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        try:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("small", "article-card__date").text.split()))
        except: pass
    return news_links, dates, found

In [ ]:
print(austria2kleinezeitung("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 1]
[PAGE NO : 2] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      1
1  2018                      0
2  2019                      0
3  2020                      0
4  2021                      0
5  2022                      0


#### 3) DIE PRESSE

In [ ]:
@scrap_data(params={'iprocess': 26, 'url': 'https://www.diepresse.com/suche?s={keyword}&p={ipage}', 'pagination': list(range(1,21))})
def austria3diepresse(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "card__content")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        try:    dates.append(" ".join(i.find("div", "card__header").get_text().split()))
        except: dates.append("")
    return news_links, dates, found

In [ ]:
print(austria3diepresse('gdpr'))

[PAGE NO : 1] [ARTICLES FOUND: 6]
[PAGE NO : 2] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      1
1  2018                      1
2  2019                      1
3  2020                      0
4  2021                      3
5  2022                      0


---

# PORTUGAL

---

#### 1) OBSERVADOR

In [ ]:
def portugal1observador(keyword):
    news_links = []
    dates = []
    iprocess = 31
    print("[WEBSITE: OBSERVADOR]")
    wd.get(f"https://observador.pt/pesquisa/?q={keyword}")    
    for ipage in list(range(21))[1:]:
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content)
        b = soup.find_all("div", "gsc-webResult gsc-result")
        print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
        for i in b:
            news_links.append(""+i.find("a").get("href"))
            try:
                dates.append(" ".join(i.find("div", "gs-visibleUrl-breadcrumb").find_all('span')[1].text.split()))
            except:
                dates.append("")
            try:
                wd.find_element_by_xpath('//*[@id="___gcse_0"]/div/div/div/div[5]/div[2]/div/div/div[2]/div/div[ipage]'.replace("ipage", str(ipage))).click()
                time.sleep(2)
            except:
                y = 0
    kamus = {"news link": news_links, "date": dates}
    return createExcelFiles(iprocess, kamus, keyword)


In [ ]:
print(portugal1observador("gdpr"))

[WEBSITE: OBSERVADOR]
[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 20]
[PAGE NO : 6] [ARTICLES FOUND: 20]
[PAGE NO : 7] [ARTICLES FOUND: 20]
[PAGE NO : 8] [ARTICLES FOUND: 20]
[PAGE NO : 9] [ARTICLES FOUND: 20]
[PAGE NO : 10] [ARTICLES FOUND: 20]
[PAGE NO : 11] [ARTICLES FOUND: 20]
[PAGE NO : 12] [ARTICLES FOUND: 20]
[PAGE NO : 13] [ARTICLES FOUND: 20]
[PAGE NO : 14] [ARTICLES FOUND: 20]
[PAGE NO : 15] [ARTICLES FOUND: 20]
[PAGE NO : 16] [ARTICLES FOUND: 20]
[PAGE NO : 17] [ARTICLES FOUND: 20]
[PAGE NO : 18] [ARTICLES FOUND: 20]
[PAGE NO : 19] [ARTICLES FOUND: 20]
[PAGE NO : 20] [ARTICLES FOUND: 20]
   year Number of Related News
0  2017                      0
1  2018                     40
2  2019                     60
3  2020                     80
4  2021                     60
5  2022                     20


#### 2) DN.PT

In [ ]:
def portugal2dn(keyword):
    news_links = []
    dates = []
    iprocess = 30
    print("[WEBSITE: DN]")
    wd.get(f"https://www.dn.pt/pesquisa.html?q={keyword}")    
    for ipage in list(range(21))[1:]:
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content)
        b = soup.find_all("div", "gsc-webResult gsc-result")
        print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
        for i in b:
            news_links.append(""+i.find("a").get("href"))
            try:
                dates.append(" ".join(i.find("div", "gs-bidi-start-align gs-snippet").text.split('...')[0].split()))
            except:
                dates.append("")
            try:
                time.sleep(1)
                wd.find_element_by_xpath('//*[@id="___gcse_0"]/div/div/div/div[5]/div[2]/div/div/div[2]/div/div[ipage]'.replace("ipage", str(ipage))).click()
                time.sleep(2)
            except:
                y = 0
    kamus = {"news link": news_links, "date": dates}
    return createExcelFiles(iprocess, kamus, keyword)

In [ ]:
print(portugal2dn("gdpr"))

[WEBSITE: DN]
[PAGE NO : 1] [ARTICLES FOUND: 9]
[PAGE NO : 2] [ARTICLES FOUND: 9]
[PAGE NO : 3] [ARTICLES FOUND: 9]
[PAGE NO : 4] [ARTICLES FOUND: 9]
[PAGE NO : 5] [ARTICLES FOUND: 9]
[PAGE NO : 6] [ARTICLES FOUND: 9]
[PAGE NO : 7] [ARTICLES FOUND: 9]
[PAGE NO : 8] [ARTICLES FOUND: 9]
[PAGE NO : 9] [ARTICLES FOUND: 9]
[PAGE NO : 10] [ARTICLES FOUND: 9]
[PAGE NO : 11] [ARTICLES FOUND: 9]
[PAGE NO : 12] [ARTICLES FOUND: 9]
[PAGE NO : 13] [ARTICLES FOUND: 9]
[PAGE NO : 14] [ARTICLES FOUND: 9]
[PAGE NO : 15] [ARTICLES FOUND: 9]
[PAGE NO : 16] [ARTICLES FOUND: 9]
[PAGE NO : 17] [ARTICLES FOUND: 9]
[PAGE NO : 18] [ARTICLES FOUND: 9]
[PAGE NO : 19] [ARTICLES FOUND: 9]
[PAGE NO : 20] [ARTICLES FOUND: 9]
   year Number of Related News
0  2017                     20
1  2018                    100
2  2019                     20
3  2020                      0
4  2021                     20
5  2022                      0


#### 3) DNOTICIAS.PT

In [ ]:
@scrap_data(params={'iprocess': 32, 'url': 'https://pesquisa.dnoticias.pt/elastic/results/?search={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def portugal3dnoticias(soup, ipage, news_links, dates):
    found = True
    b = soup.find('ul', class_='search-list-results').find_all('li')
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("span", "search-result-date").text.split()))
    return news_links, dates, found

In [ ]:
print(portugal3dnoticias("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 3]
[PAGE NO : 2] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      0
3  2020                      2
4  2021                      1
5  2022                      0


---

# ITALY

---

#### 1) REPUBBLICA.IT

In [ ]:
@scrap_data(params={'iprocess': 33, 'url': 'https://ricerca.repubblica.it/ricerca/repubblica?query={keyword}&view=repubblica&ref=HRHS&page={ipage}', 'pagination': list(range(1,21))})
def italy1repubblica(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

In [ ]:
print(italy1repubblica("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 10]
[PAGE NO : 2] [ARTICLES FOUND: 10]
[PAGE NO : 3] [ARTICLES FOUND: 10]
[PAGE NO : 4] [ARTICLES FOUND: 10]
[PAGE NO : 5] [ARTICLES FOUND: 10]
[PAGE NO : 6] [ARTICLES FOUND: 10]
[PAGE NO : 7] [ARTICLES FOUND: 10]
[PAGE NO : 8] [ARTICLES FOUND: 10]
[PAGE NO : 9] [ARTICLES FOUND: 10]
[PAGE NO : 10] [ARTICLES FOUND: 10]
[PAGE NO : 11] [ARTICLES FOUND: 10]
[PAGE NO : 12] [ARTICLES FOUND: 10]
[PAGE NO : 13] [ARTICLES FOUND: 10]
[PAGE NO : 14] [ARTICLES FOUND: 10]
[PAGE NO : 15] [ARTICLES FOUND: 10]
[PAGE NO : 16] [ARTICLES FOUND: 10]
[PAGE NO : 17] [ARTICLES FOUND: 10]
[PAGE NO : 18] [ARTICLES FOUND: 10]
[PAGE NO : 19] [ARTICLES FOUND: 10]
[PAGE NO : 20] [ARTICLES FOUND: 10]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      0
3  2020                      3
4  2021                     52
5  2022                    145


#### 2) TODAY.IT

In [ ]:
@scrap_data(params={'iprocess': 40, 'url': 'https://www.today.it/search/query/{keyword}/pag/{ipage}', 'pagination': list(range(1,21))})
def italy2today(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("span", class_='c-story__byline').find('a').get("href"))
        dates.append(" ".join(i.find("span", class_='c-story__byline').text.split(',')[-1].split()))
    return news_links, dates, found

In [ ]:
print(italy2today("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 15]
[PAGE NO : 2] [ARTICLES FOUND: 15]
[PAGE NO : 3] [ARTICLES FOUND: 15]
[PAGE NO : 4] [ARTICLES FOUND: 15]
[PAGE NO : 5] [ARTICLES FOUND: 15]
[PAGE NO : 6] [ARTICLES FOUND: 15]
[PAGE NO : 7] [ARTICLES FOUND: 15]
[PAGE NO : 8] [ARTICLES FOUND: 14]
[PAGE NO : 9] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      0
1  2018                     26
2  2019                     29
3  2020                     25
4  2021                     25
5  2022                     14


#### 3) ILPOST.IT

In [ ]:
@scrap_data(params={'iprocess': 39, 'url': 'https://www.ilpost.it/search_gcse/?q={keyword}#gsc.tab=0&gsc.q={keyword}&gsc.page={ipage}', 'pagination': list(range(1,21))})
def italy3ilpost(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", class_='gsc-webResult gsc-result')
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find('a').get("href"))
        dates.append(" ".join(i.find("div", class_='gs-visibleUrl-breadcrumb').find_all('span')[1].text.split()))
    return news_links, dates, found

In [ ]:
print(italy3ilpost("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 10]
[PAGE NO : 2] [ARTICLES FOUND: 10]
[PAGE NO : 3] [ARTICLES FOUND: 10]
[PAGE NO : 4] [ARTICLES FOUND: 10]
[PAGE NO : 5] [ARTICLES FOUND: 10]
[PAGE NO : 6] [ARTICLES FOUND: 10]
[PAGE NO : 7] [ARTICLES FOUND: 10]
[PAGE NO : 8] [ARTICLES FOUND: 10]
[PAGE NO : 9] [ARTICLES FOUND: 10]
[PAGE NO : 10] [ARTICLES FOUND: 10]
[PAGE NO : 11] [ARTICLES FOUND: 10]
[PAGE NO : 12] [ARTICLES FOUND: 10]
[PAGE NO : 13] [ARTICLES FOUND: 10]
[PAGE NO : 14] [ARTICLES FOUND: 10]
[PAGE NO : 15] [ARTICLES FOUND: 10]
[PAGE NO : 16] [ARTICLES FOUND: 10]
[PAGE NO : 17] [ARTICLES FOUND: 10]
[PAGE NO : 18] [ARTICLES FOUND: 10]
[PAGE NO : 19] [ARTICLES FOUND: 10]
[PAGE NO : 20] [ARTICLES FOUND: 10]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      0
3  2020                      0
4  2021                    180
5  2022                      0


---

# BULGARIA

---

#### 1) VESTI.BG

In [ ]:
@scrap_data(params={'iprocess': 65, 'url': 'https://www.vesti.bg/tarsene?q={keyword}', 'pagination': list(range(1,2))})
def bulgaria1vesiti(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", class_="list-item list-item-category")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find('a').get("href"))
        dates.append(" ".join(i.find("span", class_="label-time").text.split()))
    return news_links, dates, found

In [ ]:
print(bulgaria1vesiti("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 30]
   year Number of Related News
0  2017                      0
1  2018                     10
2  2019                      4
3  2020                      6
4  2021                      9
5  2022                      1


#### 2) 24CHASA.BG

In [ ]:
@scrap_data(params={'iprocess': 66, 'url': 'https://www.24chasa.bg/Search?what={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def bulgaria2chasa(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", class_="entry-short")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find('a').get("href"))
        dates.append(" ".join(i.find("span", class_="article-date").text.split('|')[-1].split()))
    return news_links, dates, found

In [ ]:
print(bulgaria2chasa("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 34]
[PAGE NO : 2] [ARTICLES FOUND: 34]
[PAGE NO : 3] [ARTICLES FOUND: 34]
[PAGE NO : 4] [ARTICLES FOUND: 2]
[PAGE NO : 5] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      1
1  2018                     46
2  2019                     33
3  2020                     11
4  2021                     10
5  2022                      3


#### 3) BLITZ.BG

In [ ]:
@scrap_data(params={'iprocess': 67, 'url': 'https://blitz.bg/search?q={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def bulgaria3blitz(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", class_="tech-news-content mt-0")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find('a').get("href"))
        dates.append(" ".join(i.find_all("p")[-1].find("span").text.split()))
    return news_links, dates, found

In [ ]:
print(bulgaria3blitz("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 15]
[PAGE NO : 2] [ARTICLES FOUND: 15]
[PAGE NO : 3] [ARTICLES FOUND: 14]
[PAGE NO : 4] [ARTICLES FOUND: 15]
[PAGE NO : 5] [ARTICLES FOUND: 14]
[PAGE NO : 6] [ARTICLES FOUND: 9]
[PAGE NO : 7] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      2
1  2018                     30
2  2019                     19
3  2020                      9
4  2021                     17
5  2022                      5


#### 4) FAKTI.BG

In [ ]:
@scrap_data(params={'iprocess': 68, 'url': 'https://fakti.bg/search?q={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def bulgaria4fakti(soup, ipage, news_links, dates):
    found = True
    b = soup.find('section', id='main').find_all("li")[:21]
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://fakti.bg"+i.find("a").get("href"))
        dates.append(" ".join(i.find("div", class_="ndtvc").find("div").text.split()))
    return news_links, dates, found

In [ ]:
print(bulgaria4fakti("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 5]
[PAGE NO : 4] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      2
1  2018                     14
2  2019                      9
3  2020                      9
4  2021                      7
5  2022                      4


---
# CROATIA
---


#### 1) INDEX.HR

In [ ]:
@scrap_data(params={'iprocess': 58, 'url': 'https://www.index.hr/trazi.aspx?upit={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def croatia1index(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="grid-items-list")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://www.index.hr"+i.find("a").get("href"))
        dates.append(" ".join(i.find("div", class_="publish-date").text.split()))
    return news_links, dates, found

In [ ]:
print(croatia1index("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 3]
[PAGE NO : 2] [ARTICLES FOUND: 3]
[PAGE NO : 3] [ARTICLES FOUND: 3]
[PAGE NO : 4] [ARTICLES FOUND: 3]
[PAGE NO : 5] [ARTICLES FOUND: 3]
[PAGE NO : 6] [ARTICLES FOUND: 3]
[PAGE NO : 7] [ARTICLES FOUND: 3]
[PAGE NO : 8] [ARTICLES FOUND: 3]
[PAGE NO : 9] [ARTICLES FOUND: 3]
[PAGE NO : 10] [ARTICLES FOUND: 3]
[PAGE NO : 11] [ARTICLES FOUND: 3]
[PAGE NO : 12] [ARTICLES FOUND: 3]
[PAGE NO : 13] [ARTICLES FOUND: 3]
[PAGE NO : 14] [ARTICLES FOUND: 3]
[PAGE NO : 15] [ARTICLES FOUND: 3]
[PAGE NO : 16] [ARTICLES FOUND: 3]
[PAGE NO : 17] [ARTICLES FOUND: 3]
[PAGE NO : 18] [ARTICLES FOUND: 3]
[PAGE NO : 19] [ARTICLES FOUND: 3]
[PAGE NO : 20] [ARTICLES FOUND: 3]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      5
3  2020                     23
4  2021                     24
5  2022                      8


#### 2) TPORTAL.HR

In [ ]:
@scrap_data(params={'iprocess': 62, 'url': 'https://www.tportal.hr/pretrazivanje?query={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def croatia2tportal(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="articlePreviewListType10")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://www.tportal.hr"+i.find("a").get("href"))
        dates.append(" ".join(i.find("h2", class_="title publishedSort").text.split()))
    return news_links, dates, found

In [ ]:
print(croatia2tportal("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 10]
[PAGE NO : 2] [ARTICLES FOUND: 10]
[PAGE NO : 3] [ARTICLES FOUND: 9]
[PAGE NO : 4] [ARTICLES FOUND: 7]
[PAGE NO : 5] [ARTICLES FOUND: 10]
[PAGE NO : 6] [ARTICLES FOUND: 7]
[PAGE NO : 7] [ARTICLES FOUND: 8]
[PAGE NO : 8] [ARTICLES FOUND: 2]
[PAGE NO : 9] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      2
1  2018                     21
2  2019                     16
3  2020                     11
4  2021                     10
5  2022                      2


#### 3) DNEVNO.HR

In [ ]:
@scrap_data(params={'iprocess': 64, 'url': 'https://www.dnevno.hr/?s={keyword}', 'pagination': list(range(1,2))})
def croatia3dnevno(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('article', class_="post")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("time", class_="date").text.split()))
    return news_links, dates, found

In [ ]:
print(croatia3dnevno("gdpr"))

[PAGE NO : 1] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      0
3  2020                      0
4  2021                      0
5  2022                      0


---
# Greece
---

#### 1) PROTOTHEMA.GR

In [ ]:
@scrap_data(params={'iprocess': 49, 'url': 'https://www.protothema.gr/ajax/Atcom.Sites.ProtoThema.Components.Search.Page/?pg={ipage}&q={keyword}', 'pagination': list(range(1,21))})
def greece1protothema(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("span", class_="update_well").text.split(',')[0].split()))
    return news_links, dates, found

In [ ]:
print(greece1protothema("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 10]
[PAGE NO : 2] [ARTICLES FOUND: 10]
[PAGE NO : 3] [ARTICLES FOUND: 10]
[PAGE NO : 4] [ARTICLES FOUND: 10]
[PAGE NO : 5] [ARTICLES FOUND: 10]
[PAGE NO : 6] [ARTICLES FOUND: 10]
[PAGE NO : 7] [ARTICLES FOUND: 10]
[PAGE NO : 8] [ARTICLES FOUND: 10]
[PAGE NO : 9] [ARTICLES FOUND: 10]
[PAGE NO : 10] [ARTICLES FOUND: 10]
[PAGE NO : 11] [ARTICLES FOUND: 10]
[PAGE NO : 12] [ARTICLES FOUND: 10]
[PAGE NO : 13] [ARTICLES FOUND: 10]
[PAGE NO : 14] [ARTICLES FOUND: 10]
[PAGE NO : 15] [ARTICLES FOUND: 10]
[PAGE NO : 16] [ARTICLES FOUND: 6]
[PAGE NO : 17] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      2
1  2018                     40
2  2019                     24
3  2020                     42
4  2021                     39
5  2022                      9


#### 2) IN.GR

In [ ]:
@scrap_data(params={'iprocess': 51, 'url': 'https://www.in.gr/search-results/3/?q={keyword}#gsc.tab=0&gsc.q={keyword}&gsc.page={ipage}', 'pagination': list(range(1,21))})
def greece2in(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="gsc-webResult gsc-result")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("div", class_="gs-bidi-start-align gs-snippet").text.split('...')[0].split()))
    return news_links, dates, found

In [ ]:
print(greece2in("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 20]
[PAGE NO : 6] [ARTICLES FOUND: 20]
[PAGE NO : 7] [ARTICLES FOUND: 20]
[PAGE NO : 8] [ARTICLES FOUND: 20]
[PAGE NO : 9] [ARTICLES FOUND: 20]
[PAGE NO : 10] [ARTICLES FOUND: 20]
[PAGE NO : 11] [ARTICLES FOUND: 20]
[PAGE NO : 12] [ARTICLES FOUND: 20]
[PAGE NO : 13] [ARTICLES FOUND: 20]
[PAGE NO : 14] [ARTICLES FOUND: 20]
[PAGE NO : 15] [ARTICLES FOUND: 20]
[PAGE NO : 16] [ARTICLES FOUND: 20]
[PAGE NO : 17] [ARTICLES FOUND: 20]
[PAGE NO : 18] [ARTICLES FOUND: 20]
[PAGE NO : 19] [ARTICLES FOUND: 20]
[PAGE NO : 20] [ARTICLES FOUND: 20]
   year Number of Related News
0  2017                      0
1  2018                      0
2  2019                      0
3  2020                    120
4  2021                    200
5  2022                     80


#### 3) NEWSIT.GR

In [ ]:
@scrap_data(params={'iprocess': 53, 'url': 'https://www.newsit.gr/page/{ipage}/?s={keyword}', 'pagination': list(range(1,21))})
def greece3newsit(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('article', class_="blog-article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("time", class_="entry-date published").text.split()))
    return news_links, dates, found

In [ ]:
print(greece3newsit("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 1]
[PAGE NO : 6] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      0
1  2018                     21
2  2019                     22
3  2020                     20
4  2021                     15
5  2022                      2


#### 4) DIKAILOGIKA.GR

In [ ]:
@scrap_data(params={'iprocess': 52, 'url': 'https://www.dikaiologitika.gr/site/search/{keyword}?searchword={keyword}&start={ipage}', 'pagination': list(map(lambda x: x*20,range(21)))})
def greece4dikaiologitika(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="itemContainer")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://www.dikaiologitika.gr"+i.find("a").get("href"))
        dates.append(" ".join(i.find("span", class_="genericItemDateCreated").text.split('-')[0].split()))
    return news_links, dates, found

In [ ]:
print(greece4dikaiologitika("gdpr"))

[PAGE NO : 0] [ARTICLES FOUND: 20]
[PAGE NO : 20] [ARTICLES FOUND: 20]
[PAGE NO : 40] [ARTICLES FOUND: 20]
[PAGE NO : 60] [ARTICLES FOUND: 20]
[PAGE NO : 80] [ARTICLES FOUND: 20]
[PAGE NO : 100] [ARTICLES FOUND: 20]
[PAGE NO : 120] [ARTICLES FOUND: 20]
[PAGE NO : 140] [ARTICLES FOUND: 20]
[PAGE NO : 160] [ARTICLES FOUND: 13]
[PAGE NO : 180] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                     20
1  2018                     39
2  2019                     26
3  2020                     30
4  2021                     43
5  2022                     14


---
# SLOVAKIA
---

#### 1) TOPKY.SK

In [ ]:
@scrap_data(params={'iprocess': 44, 'url': 'https://www.topky.sk/search/{keyword}/{ipage}/', 'pagination': list(range(1,21))})
def slovakia1topky(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="entry")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://www.topky.sk"+i.find("a").get("href"))
        dates.append(" ".join(i.find("p", class_="time").text.split(' ')[0].split()))
    return news_links, dates, found

In [ ]:
print(slovakia1topky("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 10]
[PAGE NO : 2] [ARTICLES FOUND: 10]
[PAGE NO : 3] [ARTICLES FOUND: 10]
[PAGE NO : 4] [ARTICLES FOUND: 10]
[PAGE NO : 5] [ARTICLES FOUND: 10]
[PAGE NO : 6] [ARTICLES FOUND: 10]
[PAGE NO : 7] [ARTICLES FOUND: 10]
[PAGE NO : 8] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      2
1  2018                     23
2  2019                     15
3  2020                     14
4  2021                     13
5  2022                      3


#### 2) TERAZ.SK

In [ ]:
@scrap_data(params={'iprocess': 47, 'url': 'https://www.teraz.sk/search?q={keyword}&p={ipage}', 'pagination': list(range(1,21))})
def slovakia2teraz(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="mediaListing-item")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append("https://www.teraz.sk"+i.find("a").get("href"))
        dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

In [ ]:
print(slovakia2teraz("gdpr"))

[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 20]
[PAGE NO : 6] [ARTICLES FOUND: 20]
[PAGE NO : 7] [ARTICLES FOUND: 20]
[PAGE NO : 8] [ARTICLES FOUND: 20]
[PAGE NO : 9] [ARTICLES FOUND: 20]
[PAGE NO : 10] [ARTICLES FOUND: 13]
[PAGE NO : 11] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      7
1  2018                     75
2  2019                     52
3  2020                     24
4  2021                     26
5  2022                      9


#### 3) AKTUALITY.SK

In [ ]:
@scrap_data(params={'iprocess': 42, 'url': 'https://www.aktuality.sk/vyhladavanie/{ipage}/?search%5Btext%5D={keyword}&search%5Bzdroj%5D=spravy', 'pagination': list(range(1,21))})
def slovakia3aktuality(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('li', class_="article-item list-item")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        news_links.append(""+i.find("a").get("href"))
        dates.append(" ".join(i.find("span", class_="article-time").text.split(' ')[0].split()))
    return news_links, dates, found

In [ ]:
print(slovakia3aktuality('gdpr'))

---

# OUTPUT 

---

Each time, one will run the function for a certain country, it will output two excel files for each keyword:

- number of news articles found for each keyword per year (2017-2022),
- links of news articles found with date (for cross-checks).

Naming conventions for the files that you will find in the folder 'results' > country_name > keywordX, keywordY, etc.:

country + news site: links for cross-checking

"keyword" + country + news site: statistics
